In [45]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [46]:
print("Num GPUs Available: ",
      len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [47]:
TRAIN_DIR = './dataset/sdd/train/'
TEST_DIR = './dataset/sdd/test/'
IMG_SIZE = (150, 150)
BATCH_SIZE = 128
SEED = 1337

In [48]:
# Training generator
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_ds = train_datagen.flow_from_directory(TRAIN_DIR,
                                             target_size=IMG_SIZE,
                                             batch_size=BATCH_SIZE,
                                             seed=SEED,
                                             shuffle=True,
                                             class_mode='categorical')

# Test generator
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_ds = test_datagen.flow_from_directory(TEST_DIR,
                                           target_size=IMG_SIZE,
                                           batch_size=BATCH_SIZE,
                                           seed=SEED,
                                           shuffle=False,
                                           class_mode='categorical')

train_num = train_ds.samples
test_num = test_ds.samples

Found 16229 images belonging to 120 classes.
Found 4111 images belonging to 120 classes.


In [49]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(GlobalAvgPool2D())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(120))
model.add(Activation('softmax'))

In [50]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
activation_48 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_49 (Activation)   (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 36, 36, 64)       

In [ ]:
EPOCHS = 1

checkpoint_path = "checkpoint/dog/cp.ckpt"

cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              save_weights_only=True,
                              save_best_only=True,
                              mode='min',
                              verbose=1)

history = model.fit_generator(train_ds,
                              steps_per_epoch=train_num // BATCH_SIZE,
                              epochs=EPOCHS,
                              validation_data=test_ds,
                              validation_steps=test_num // BATCH_SIZE,
                              callbacks=[cp_callback])

126/126 [==============================] - ETA: 0s - loss: 1.9640 - acc: 0.4566